In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
sinkerDL = pd.read_csv('DraftLeagueSinker.csv')
sinkerMLB = pd.read_csv('MLBSinker.csv')

In [3]:
sinkerDL['league'] = 'Draft League'
sinkerMLB['league'] = 'MLB'

In [4]:
sinkerDL = sinkerDL.astype(str)
sinkerMLB = sinkerMLB.astype(str)

In [5]:
pitchersDL = list(set(sinkerDL['pitcher']))

In [6]:
slider = pd.concat([sinkerDL, sinkerMLB])

In [7]:
def combined_features(row):
    return row['avgvelo']+" "+row['avgspinrate']+" "+row['avghorzbreak']+" "+row['avgvertbreak']+" "+row['avgeffectvelo']

In [8]:
#fig, ax = plt.subplots(figsize=(100,100))
#sns.heatmap(cosine_sim,cmap='magma',ax=ax)

In [9]:
def get_index_from_pitcher(pitcher):
    return sinker[sinker.pitcher == pitcher].index[0]

In [10]:
def get_pitcher_from_index(index):
    return sinker[sinker.index == index].values[0][0]

In [11]:
for x in pitchersDL:
    
    try:

        sinker = pd.concat([sinkerDL, sinkerMLB])

        sinker["combined_features"] = sinker.apply(combined_features, axis =1)

        cv = CountVectorizer()
        count_matrix = cv.fit_transform(sinker["combined_features"])

        cosine_sim = cosine_similarity(count_matrix)

        pitcher_like = x
        sinker = sinker[(sinker['pitcher'] == pitcher_like) | (sinker['league'] == 'MLB')]

        player_index = get_index_from_pitcher(pitcher_like)

        similar_pitchers = list(enumerate(cosine_sim[player_index]))

        sorted_similar_pitchers = sorted(similar_pitchers, key=lambda x:x[1], reverse=True)

        i=0

        for pitcher in sorted_similar_pitchers:
            print(get_pitcher_from_index(pitcher[0]),", Cosine Sim: ", sorted_similar_pitchers[i][1])
            i=i+1
            if i>1:
                break
                
    except IndexError:
        print(x, ', No Similarity')
        pass

Johnson, Tyler , Cosine Sim:  0.9999999999999999
Familia, Jeurys , Cosine Sim:  0.38138503569823695
Knoll, Brendan , Cosine Sim:  0.9999999999999999
May, Trevor , Cosine Sim:  0.30000000000000004
Ramos, Jean Paul , Cosine Sim:  0.9999999999999999
García, Yimi , Cosine Sim:  0.30000000000000004
Rodriguez, Joe Joe , Cosine Sim:  0.9999999999999999
Familia, Jeurys , Cosine Sim:  0.38138503569823695
Sapp, Eric , Cosine Sim:  0.9999999999999999
Ureña, José , Cosine Sim:  0.3508232077228117
Cherry, Derrick , Cosine Sim:  0.9999999999999999
Cherry, Derrick , No Similarity
Murphy, Shane , Cosine Sim:  0.9999999999999999
White, Mitch , Cosine Sim:  0.30000000000000004
Kiritsis, Daniel , Cosine Sim:  1.0
Szapucki, Thomas , Cosine Sim:  0.42640143271122083
Diedrich, Sayer , Cosine Sim:  0.9999999999999999
Eshelman, Thomas , Cosine Sim:  0.31622776601683794
Yamane, Tiaga , Cosine Sim:  0.9999999999999999
Farrell, Luke , Cosine Sim:  0.30000000000000004
Walling, Andrew , Cosine Sim:  0.999999999999

Mineo, Ricky , Cosine Sim:  1.0000000000000002
Mineo, Ricky , No Similarity
Bosch, Carter , Cosine Sim:  1.0000000000000002
Bosch, Carter , No Similarity
Steward, Casey , Cosine Sim:  0.9999999999999999
Sharp, Sterling , Cosine Sim:  0.31622776601683794
Olson, Mason , Cosine Sim:  0.9999999999999999
Loáisiga, Jonathan , Cosine Sim:  0.30000000000000004
Velazquez, Ryan , Cosine Sim:  0.9999999999999999
Madero, Luís , Cosine Sim:  0.4
Modugno, John , Cosine Sim:  0.9999999999999999
Szapucki, Thomas , Cosine Sim:  0.33541019662496846
Parisi, Jack , Cosine Sim:  0.9999999999999999
Effross, Scott , Cosine Sim:  0.3585685828003181
Wall, Chris , Cosine Sim:  0.9999999999999999
Bradley, Archie , Cosine Sim:  0.31622776601683794
Wojtarowicz, Andre , Cosine Sim:  1.0000000000000002
Hand, Brad , Cosine Sim:  0.31622776601683794
Oswald, Wesley , Cosine Sim:  0.9999999999999999
Szapucki, Thomas , Cosine Sim:  0.33541019662496846
Bravo, Zach , Cosine Sim:  0.9999999999999999
Bashlor, Tyler , Cosine 